# T5 - Juan Luis Baldelomar Cabrera

In [1]:
# os
import random

# NLP and numpy
import nltk 
import numpy as np
import nltk
from nltk.probability import FreqDist
from nltk import TweetTokenizer
from nltk.corpus import stopwords
import pandas as pd

# NGrams File
from NGrams import NGramBuilder
from NGrams import NGramNeuralModel

# torch
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.nn import functional as F

# metrics
from sklearn.metrics import accuracy_score as accuracy

In [2]:
seed = 1111
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False

# Load Data

In [3]:
def load_data(filename, labels_filename):
    file = open(filename, 'r')
    labels_file = open(labels_filename, 'r')
    tweets = file.read()
    labels = labels_file.read()
    documents = tweets.split('\n')
    labels = labels.split('\n')
    documents.pop(-1)
    labels.pop(-1)
    file.close()
    labels_file.close()
    return documents, labels

In [4]:
documents, labels = load_data('data/mex_train.txt', 'data/mex_train_labels.txt')
val_documents, val_labels = load_data('data/mex_val.txt', 'data/mex_val_labels.txt')

# String and Doc Utilities

In [5]:
def bold_string(string):
    return '\033[1m' + string + '\033[0m '

def print_doc(doc:list, end=' ', stop=-1):
    stop = len(doc) if stop is None else stop
    for token in doc[:stop]:
        print(token, end=end)
    print('')

# NGram Builder Class

La clase para construir n-gramas se encuentra en el archivo NGrams.py. En esta también se encuentra la clase NGramNeuralModel que se encarga de mezclar las funcionalidades de un objeto NgramBuilder y el modelo neuronal entrenado. Entre estas funcionalidades se encuentra la estimación de probabilidades, generación de secuencias, muestreo de palabras, etc. A contitnuación tenemos unos ejemplos de como construir los ngramas.

In [32]:
ngram_builder = NGramBuilder()
ngram_docs, ngram_labels = ngram_builder.fit(documents, N=4)
val_ngram_docs, val_ngram_labels = ngram_builder.transform(val_documents)
ngram_builder.emb_matrix.shape

(10000, 256)

In [10]:
doc = ngram_builder.inverse(ngram_labels)
print_doc(doc[:30])
del(ngram_builder);

lo peor de todo es que no me dan por un tiempo y luego vuelven estoy hasta la verga de estl </s> a la vga no seas mamón 45 


# Dataset

In [6]:
def get_datasets(ngram_builder, N, train_docs, val_docs, batch_size=64, num_workers=2):
    ngram_docs, ngram_labels = ngram_builder.fit(documents, N=N)
    val_ngram_docs, val_ngram_labels = ngram_builder.transform(val_documents)
    
    train_ds = TensorDataset(torch.tensor(ngram_docs, dtype=torch.int64), torch.tensor(ngram_labels, dtype=torch.int64))
    train_loader = DataLoader(train_ds, shuffle=True, batch_size=batch_size, num_workers=num_workers)

    val_ds = TensorDataset(torch.tensor(val_ngram_docs, dtype=torch.int64), torch.tensor(val_ngram_labels, dtype=torch.int64))
    val_loader = DataLoader(val_ds, shuffle=False, batch_size=batch_size, num_workers=num_workers)
    
    return train_ds, train_loader, val_ds, val_loader

# Test Syntactical and Morphological Structures 

In [7]:
from itertools import permutations

def get_perms(tokens):
    perms = set(permutations(tokens))
    return list(perms)

def test_structures(tokens, ngram_model):
    perms = get_perms(tokens)
    likelihoods = [(ngram_model.estimate_prob(' '.join(perm), use_gpu=use_gpu), ' '.join(perm)) for perm in perms]
    likelihoods = sorted(likelihoods, reverse=True)
    for l, sentence in likelihoods:
        print(sentence)
        print('likelihood: ', l, end='\n\n')

# Ejercicio 1

## Char NGram

In [6]:
# function to call after normal tokenization to get each word as a document, i.e <s> word1 </s>, <s> word2 </s>, ...
def char_postprocess(documents):
    return [[c for c in word] for doc in documents for word in doc]

# tokenize documents char by char so you can add <s> and </s> at end of each doc
def char_tokenizer(doc):
    return [char for char in doc]

In [7]:
char_ngram_builder = NGramBuilder(tokenizer=char_tokenizer, d_model=100)
ngram_docs, ngram_labels = char_ngram_builder.fit(documents, N=6)
val_ngram_docs, val_ngram_labels = char_ngram_builder.transform(val_documents)

In [66]:
words = char_ngram_builder.inverse(ngram_labels)
print_doc(words[:100], end='', stop=-1)

lo peor de todo es que no me dan por un tiempo y luego vuelven estoy hasta la verga de estl</s>a la vg


## Neural Language Model

In [103]:
class BengioModel(nn.Module):
    def __init__(self, N, voc_size, d_model, hidden_size=128, emb_mat=None, dropout=0.1):
        
        super(BengioModel, self).__init__()
        # parameters
        self.N           = N
        self.d_model     = d_model
        self.voc_size    = voc_size
        self.hidden_size = hidden_size
        
        # Matriz entrenable de embeddings, tamaño vocab_size x Ngram.d_model
        self.embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(emb_mat), freeze=False)
        
        # fully connected layers
        self.fc1 = nn.Linear(d_model * (N-1), hidden_size)
        self.fc2 = nn.Linear(hidden_size, voc_size, bias=False)
        
        # dropout
        self.drop = nn.Dropout(dropout)
        
    
    def forward(self, input_seq):
        # Calcula el embedding para cada palabra.
        x = self.embeddings(input_seq)
        x = x.view(-1, (self.N-1) * self.d_model)
        x = self.fc1(x)
        x = self.drop(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [8]:
def get_preds(raw_logit):
    probs = F.softmax(raw_logit.detach(), dim=1)
    y_pred = torch.argmax(probs, dim=1).cpu().numpy()
    return y_pred

In [9]:
def get_probs(raw_logit):
    probs = F.softmax(raw_logit.detach(), dim=1)
    return probs.cpu().numpy()

## Eval

In [10]:
def eval_model(data, model, gpu=False):
    preds, targets = [], []
    with torch.no_grad():
        for inputs, labels in data:
            if gpu:
                # move inputs to gpu
                inputs = inputs.cuda()
            
            # compute output predictions    
            output = model(inputs)
            batch_preds = get_preds(output)
            # append preds and targets
            preds.append(batch_preds)
            targets.append(labels.numpy())
    
    # remove batch dimension
    preds = [p for batch_pred in preds for p in batch_pred]
    targets = [t for batch_tar in targets for t in batch_tar]
    return accuracy(preds, targets)

In [11]:
def checkpoint(state, path, val_acc, best_metric, override=False):
    if val_acc > best_metric or override: 
        print('Storing best model to {0}. Current acc: {1}, last best metric: {2}'.format(path, val_acc, best_metric))
        torch.save(state, path)

## Hyperparameters

In [15]:
# model hyperparameters
voc_size = char_ngram_builder.voc_size
N = char_ngram_builder.N
d_model = char_ngram_builder.d_model

# optimizer hyperparameters
lr = 2.3e-1 
epochs = 100
patience = epochs//5

# scheduler hyperparameters
lr_patience = 10
lr_factor = 0.5

# gpu available?
use_gpu = torch.cuda.is_available()

# build model and move to gpu if possible
model = BengioModel(N=N, voc_size=voc_size, d_model=d_model, hidden_size=200, emb_mat=char_ngram_builder.emb_matrix)
if use_gpu:
    model = model.cuda()
    
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer,
                'min',
                patience = lr_patience,
                verbose=True,
                factor = lr_factor
            )

criterion = nn.CrossEntropyLoss()

## Training

In [16]:
train_ds, train_loader, val_ds, val_loader = get_datasets(char_ngram_builder, 6, documents, val_documents, batch_size=256, num_workers=1)

In [72]:
best_metric = 0
last_metric = 0
val_metrics = []
counter = 0

for epoch in range(epochs):
    print('epoch: ', 1 + epoch)
    epoch_metrics = []
    epoch_losses = []
    model.train()
    for inputs, targets in train_loader:
        if use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()

        # feed model and get loss
        output = model(inputs)
        loss = criterion(output, targets)
        epoch_losses.append(loss.item())

        # metric with train dataset
        preds = get_preds(output)
        epoch_metrics.append(accuracy(preds, targets.cpu().numpy()))

        # step to optimize 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # close for each step

    # get metric for training set
    model.eval()
    train_acc = np.mean(epoch_metrics)
    val_acc = eval_model(val_loader, model, use_gpu)
    val_metrics.append(val_acc)

    # print metrics
    print('train accuracy mean: ', train_acc)
    print('validation accuracy: ', val_acc)
    print('mean loss: ', np.mean(epoch_losses))

    # store model if necessary
    state = {
                'epoch' : epoch + 1,
                'optimizer': optimizer.state_dict(),
                'model': model.state_dict(),
                'scheduler': scheduler.state_dict(),
                'best_metric': best_metric
            }
    checkpoint(state, 'char_best_model', val_acc, best_metric)

    # patience and last_metric and best_metric update
    last_metric = val_acc
    counter = counter + 1 if last_metric <= best_metric else 0
    best_metric = val_acc if val_acc > best_metric else best_metric

    # check if patience run out
    if counter >= patience:
        break
# close for each epoch

epoch:  1
train accuracy mean:  0.3443141102940012
validation accuracy:  0.4061286064491797
mean loss:  2.297941785050635
Storing best model to char_best_model. Current acc: 0.4061286064491797, last best metric: 0
epoch:  2
train accuracy mean:  0.4265518076082308
validation accuracy:  0.4336413350933434
mean loss:  1.960996209173008
Storing best model to char_best_model. Current acc: 0.4336413350933434, last best metric: 0.4061286064491797
epoch:  3
train accuracy mean:  0.44810113517559286
validation accuracy:  0.44148595134829344
mean loss:  1.875410953305868
Storing best model to char_best_model. Current acc: 0.44148595134829344, last best metric: 0.4336413350933434
epoch:  4
train accuracy mean:  0.46165679848009633
validation accuracy:  0.4551386007920045
mean loss:  1.8260459936045255
Storing best model to char_best_model. Current acc: 0.4551386007920045, last best metric: 0.44148595134829344
epoch:  5
train accuracy mean:  0.47037113489838267
validation accuracy:  0.46671695266

# Load Model

In [17]:
load_state = torch.load('char_best_model')
model.load_state_dict(load_state['model'])

<All keys matched successfully>

In [18]:
model.eval()
eval_model(val_loader, model, use_gpu)

0.5226664152366585

## Test NGram Neural Model

In [19]:
NGramModel = NGramNeuralModel(char_ngram_builder, model)

### Sequence Generation

In [20]:
seq = NGramModel.generate_sequence(use_gpu=use_gpu)
print_doc(seq, end='')

<s><s><s><s><s>padre por 35 el que se valer tal al estoy loca jajajaja de lefandarse no tengo monzaba que con


In [22]:
seq = NGramModel.generate_sequence(use_gpu=use_gpu, max_length=300)
print_doc(seq, end='')

<s><s><s><s><s>copanaduerdos aún así juegas a togerdistaso ajenatacianta que se volvera putas cosalo idea


In [77]:
seq = NGramModel.generate_sequence(use_gpu=use_gpu, max_length=300)
print_doc(seq, end='')

<s><s><s><s><s>el robo son por atuger mañana de puto en no dejones puto así solo madre tonen no de verga que te wará el arme este el tiemposin no lo tembici el poctando maricónsiso me de que marica de verga de páginas del my pagudite y ella a sentalya jajajajajajaja


### Sequence Probability Estimation

In [23]:
NGramModel.estimate_prob('vete a la verga', use_gpu=use_gpu)

-4.710024

In [24]:
NGramModel.estimate_prob('a la vete verga', use_gpu=use_gpu)

-13.1000185

In [25]:
NGramModel.estimate_prob('esos hijos de la chingada', use_gpu=use_gpu)

-16.086407

In [60]:
NGramModel.estimate_prob('esos chingada de los hijos', use_gpu=use_gpu)

-21.678864

In [58]:
NGramModel.estimate_prob('estuvieron', use_gpu=use_gpu)

-4.1125507

In [57]:
NGramModel.estimate_prob('estuveiron', use_gpu=use_gpu)

-20.756233

In [61]:
NGramModel.estimate_prob('vete alv', use_gpu=use_gpu)

-6.3073807

In [62]:
NGramModel.estimate_prob('vete avl', use_gpu=use_gpu)

-16.073044

In [63]:
NGramModel.estimate_prob('veet avl', use_gpu=use_gpu)

-15.13043

## Permutations

In [33]:
test_structures(list('amor '), NGramModel)

r o   a m
likelihood:  -8.757793

  r o a m
likelihood:  -9.794529

o r   a m
likelihood:  -10.665074

m o   a r
likelihood:  -11.240616

r   o a m
likelihood:  -11.738606

  m o a r
likelihood:  -11.784495

r m o   a
likelihood:  -12.62795

r a   o m
likelihood:  -12.659768

r m o a  
likelihood:  -12.839628

o m   a r
likelihood:  -12.941523

a r   o m
likelihood:  -12.956074

m   o a r
likelihood:  -13.132235

r m   o a
likelihood:  -13.270688

m r o a  
likelihood:  -13.477239

a r o m  
likelihood:  -13.480266

m r o   a
likelihood:  -13.740321

r o m a  
likelihood:  -13.768693

r m a   o
likelihood:  -13.921881

o m a   r
likelihood:  -13.964831

  r o m a
likelihood:  -14.001951

r m   a o
likelihood:  -14.005707

m r   a o
likelihood:  -14.123345

m r   o a
likelihood:  -14.132148

r   a o m
likelihood:  -14.150971

a r o   m
likelihood:  -14.210142

a m o   r
likelihood:  -14.219949

r o a m  
likelihood:  -14.234589

r   o m a
likelihood:  -14.533393

o r a   m
likelihood:  

### Perplexity

In [34]:
NGramModel.perplexity(val_documents, use_gpu=use_gpu)

5.051324235262274

## Conclusión

Podemos ver como la perplejidad de este modelo es más baja que la de los ngramas de palabras, pero esto es porque en general las posibilidades para continuar una secuencia son mucho menores, debido a que el tamaño del vocabulario en este caso son los diferentes caracteres que son mucho menos que el tamaño del vocabulario utilizado con palabras. Por lo tanto es de esperar que la perplejidad sea menor. 

Podemos ver a través de la generación de secuencias como logra construir algunas palabras correctamente, sin embargo no existe una estructura sintáctica de fondo, y no se esperaba que con este nivel de granularidad se pudiera modela la semántica o sintaxis de una oración.   

# Datasets to be used in the rest of Notebook

In [12]:
ngram_builder = NGramBuilder(d_model=100)
ngram_docs, ngram_labels = ngram_builder.fit(documents, N=4)

In [13]:
train_ds, train_loader, val_ds, val_loader = get_datasets(ngram_builder, 4, documents, val_documents, batch_size=64, num_workers=2)

# Ejercicio 2

Para este ejercicio entrenaremos dos modelos. Un modelo de tetragramas sin una tabla de embeddings pre-entrenados, y otro modelo de tetragramas con un modelo de embeddings pre-entrenado. 

Cabe resaltar que para todos los siguientes ejercicios se utilizó un vocabulario con los 10000 tokens más frecuentes.

## Most Similar Words

Esta función nos ayudara a obtener las palabras más similares a una palabra en específico según el modelo de embeddings obtenido al final del entrenamiento.

In [41]:
def most_similar_to(word, ngram_builder, embeddings, N):
    # get word id
    word_id = ngram_builder.get_ids([word])[0]
    word_rep = embeddings[word_id]
    # get norms to normalize later
    embeddings_norm = np.linalg.norm(embeddings, axis=1)
    word_norm = embeddings_norm[word_id]
    # sim distance
    distances = np.dot(word_rep, embeddings.T)
    # normalize distances (cos distance)
    distances = np.squeeze(distances/(word_norm * embeddings_norm))
    
    # most similar word is surely the word itself, so ignore the most similar
    return np.argsort(distances)[-(N+1):-1]

## No Embeddings Model

In [ ]:
ngram_builder = NGramBuilder(d_model=100)
ngram_docs, ngram_labels = ngram_builder.fit(documents, N=4)

In [135]:
# model hyperparameters
voc_size = ngram_builder.voc_size
N = ngram_builder.N
d_model = ngram_builder.d_model

# optimizer hyperparameters
lr = 2.3e-1 
epochs = 100
patience = epochs//5

# scheduler hyperparameters
lr_patience = 10
lr_factor = 0.5

# gpu available?
use_gpu = torch.cuda.is_available()

# build model and move to gpu if possible
model = BengioModel(N=N, voc_size=voc_size, d_model=d_model, hidden_size=200, emb_mat=ngram_builder.emb_matrix)
if use_gpu:
    model = model.cuda()

# optimizer and scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer,
                'min',
                patience = lr_patience,
                verbose=True,
                factor = lr_factor
            )

criterion = nn.CrossEntropyLoss()

In [136]:
best_metric = 0
last_metric = 0
val_metrics = []
counter = 0

for epoch in range(epochs):
    print('epoch: ', 1 + epoch)
    epoch_metrics = []
    epoch_losses = []
    model.train()
    for inputs, targets in train_loader:
        if use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()

        # feed model and get loss
        output = model(inputs)
        loss = criterion(output, targets)
        epoch_losses.append(loss.item())

        # metric with train dataset
        preds = get_preds(output)
        epoch_metrics.append(accuracy(preds, targets.cpu().numpy()))

        # step to optimize 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # close for each step

    # get metric for training set
    model.eval()
    train_acc = np.mean(epoch_metrics)
    val_acc = eval_model(val_loader, model, use_gpu)
    val_metrics.append(val_acc)

    # print metrics
    print('train accuracy mean: ', train_acc)
    print('validation accuracy: ', val_acc)
    print('mean loss: ', np.mean(epoch_losses))

    # store model if necessary
    state = {
                'epoch' : epoch + 1,
                'optimizer': optimizer.state_dict(),
                'model': model.state_dict(),
                'scheduler': scheduler.state_dict(),
                'best_metric': best_metric
            }
    checkpoint(state, 'no_embeddings_best_model', val_acc, best_metric)

    # patience and last_metric and best_metric update
    last_metric = val_acc
    counter = counter + 1 if last_metric <= best_metric else 0
    best_metric = val_acc if val_acc > best_metric else best_metric

    # check if patience run out
    if counter >= patience:
        break
# close for each epoch

epoch:  1
train accuracy mean:  0.061187744140625
validation accuracy:  0.06553990610328639
mean loss:  6.567409899706642
Storing best model to no_embeddings_best_model. Current acc: 0.06553990610328639, last best metric: 0
epoch:  2
train accuracy mean:  0.09185936337425595
validation accuracy:  0.10375586854460093
mean loss:  6.168204473952453
Storing best model to no_embeddings_best_model. Current acc: 0.10375586854460093, last best metric: 0.06553990610328639
epoch:  3
train accuracy mean:  0.10659886920262897
validation accuracy:  0.12647887323943663
mean loss:  5.965309233404696
Storing best model to no_embeddings_best_model. Current acc: 0.12647887323943663, last best metric: 0.10375586854460093
epoch:  4
train accuracy mean:  0.11321197994171628
validation accuracy:  0.12150234741784037
mean loss:  5.819347424432635
epoch:  5
train accuracy mean:  0.12097361731150795
validation accuracy:  0.08666666666666667
mean loss:  5.689494291010003
epoch:  6
train accuracy mean:  0.126211

## Load Model

In [149]:
load_state = torch.load('no_embeddings_best_model')
model.load_state_dict(load_state['model'])
model.eval()
NGramModel = NGramNeuralModel(ngram_builder, model)
print('accuracy in val: ', eval_model(val_loader, model, use_gpu))

accuracy in val:  0.1584037558685446


## Most Similar Words

In [138]:
word = 'chinga'
indexes = most_similar_to(word, ngram_builder, model.embeddings.weight.detach().cpu().numpy(), 10)
print('Most Similar to: ', bold_string(word))
ngram_builder.inverse(indexes)

Most Similar to:  chinga 


['aterrador',
 'quééé',
 'escoja',
 'enseña',
 '#politicos',
 '⭐',
 'imagen',
 'verán',
 '👎🏼',
 'pongas']

In [64]:
word = 'amor'
indexes = most_similar_to(word, ngram_builder, model.embeddings.weight.detach().cpu().numpy(), 10)
print('Most Similar to: ', bold_string(word))
ngram_builder.inverse(indexes)

Most Similar to:  amor 


['puntome',
 'narcotrafico',
 'entender',
 'cool',
 'sobredosis',
 '#milesheizer',
 'clientes',
 'ves',
 'drastico',
 'reparaciones']

In [65]:
word = 'verga'
indexes = most_similar_to(word, ngram_builder, model.embeddings.weight.detach().cpu().numpy(), 10)
print('Most Similar to: ', bold_string(word))
ngram_builder.inverse(indexes)

Most Similar to:  verga 


['clases',
 '#btw',
 'hablen',
 'abdomen',
 'blog',
 'tardas',
 'rayo',
 'cabroncito',
 'inundo',
 'cerebro']

Podemos resaltar como la tabla de embeddings entrenados desde 0 no parece resaltar este concepto de agrupar términos similares a través de los embeddings. 

## Perplexity

In [150]:
NGramModel.perplexity(val_documents, use_gpu=use_gpu)

352.9230342943041

# Embeddings Model

## Load Embeddings

In [15]:
class Embeddings:
    def __init__(self, filename):
        self.embeddings = {}
        with open(filename, 'r') as file:
            for line in file:
                values = line.split()
                word, rep = values[0], np.array(list(map(float, values[1:])))
                self.embeddings[word] = rep
                self.d_model = len(rep)
            
    def __getitem__(self, index):
        return self.embeddings.get(index)

In [16]:
embeddings = Embeddings('data/word2vec_col.txt')
embeddings['de']

array([-1.64168 ,  1.447671, -2.283216, -1.965226, -0.222943,  5.105217,
       -0.120701, -0.126822, -3.177338, -3.454396, -0.943083, -0.094476,
       -1.18936 , -0.812092, -2.572975, -0.613877, -2.311841,  1.05097 ,
        5.634725, -5.827006,  1.237639,  1.071621,  3.822072,  2.395414,
        0.169883,  3.256835,  2.897348,  3.274827, -2.936382,  0.272003,
       -1.029505, -2.617288, -1.807143,  1.737624,  0.33913 ,  3.93293 ,
        1.571361, -4.100074,  4.156816,  1.162366, -0.552316, -0.585887,
       -4.767187,  0.253338, -1.124162, -0.115079, -5.606624,  2.976579,
        4.426022,  1.019932,  3.76072 , -2.298347,  4.416567, -1.383988,
       -1.862506,  0.399053, -1.09689 , -2.28599 ,  2.992802,  0.044008,
        3.762375, -6.523126,  0.621278,  2.641829, -1.924327, -1.141184,
       -3.831767,  0.549591,  2.260839, -1.318358, -1.134662, -3.788221,
       -0.775024,  3.956695, -3.579425, -4.423733,  4.505686,  0.719133,
       -1.399557,  3.097209,  0.107541,  2.829867, 

In [140]:
ngram_builder = NGramBuilder(embeddings=embeddings)
ngram_docs, ngram_labels = ngram_builder.fit(documents, N=4)

## Hyperparameters

In [142]:
# model hyperparameters
voc_size = ngram_builder.voc_size
N = ngram_builder.N
d_model = ngram_builder.d_model

# optimizer hyperparameters
lr = 2.3e-1 
epochs = 100
patience = epochs//5

# scheduler hyperparameters
lr_patience = 10
lr_factor = 0.5

# gpu available?
use_gpu = torch.cuda.is_available()

# build model and move to gpu if possible
model = BengioModel(N=N, voc_size=voc_size, d_model=d_model, hidden_size=200, emb_mat=ngram_builder.emb_matrix)
if use_gpu:
    model = model.cuda()
    
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer,
                'min',
                patience = lr_patience,
                verbose=True,
                factor = lr_factor
            )

criterion = nn.CrossEntropyLoss()

## Train Embeddings Model

In [144]:
best_metric = 0
last_metric = 0
val_metrics = []
counter = 0

for epoch in range(epochs):
    print('epoch: ', 1 + epoch)
    epoch_metrics = []
    epoch_losses = []
    model.train()
    for inputs, targets in train_loader:
        if use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()

        # feed model and get loss
        output = model(inputs)
        loss = criterion(output, targets)
        epoch_losses.append(loss.item())

        # metric with train dataset
        preds = get_preds(output)
        epoch_metrics.append(accuracy(preds, targets.cpu().numpy()))

        # step to optimize 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # close for each step

    # get metric for training set
    model.eval()
    train_acc = np.mean(epoch_metrics)
    val_acc = eval_model(val_loader, model, use_gpu)
    val_metrics.append(val_acc)

    # print metrics
    print('train accuracy mean: ', train_acc)
    print('validation accuracy: ', val_acc)
    print('mean loss: ', np.mean(epoch_losses))

    # store model if necessary
    state = {
                'epoch' : epoch + 1,
                'optimizer': optimizer.state_dict(),
                'model': model.state_dict(),
                'scheduler': scheduler.state_dict(),
                'best_metric': best_metric
            }
    checkpoint(state, 'embeddings_best_model', val_acc, best_metric)

    # patience and last_metric and best_metric update
    last_metric = val_acc
    counter = counter + 1 if last_metric <= best_metric else 0
    best_metric = val_acc if val_acc > best_metric else best_metric

    # check if patience run out
    if counter >= patience:
        break
# close for each epoch

epoch:  1
train accuracy mean:  0.0976160443018353
validation accuracy:  0.08610328638497652
mean loss:  6.43358594737947
Storing best model to embeddings_best_model. Current acc: 0.08610328638497652, last best metric: 0
epoch:  2
train accuracy mean:  0.11321149553571429
validation accuracy:  0.14178403755868543
mean loss:  5.907127746691306
Storing best model to embeddings_best_model. Current acc: 0.14178403755868543, last best metric: 0.08610328638497652
epoch:  3
train accuracy mean:  0.11843000139508929
validation accuracy:  0.1115492957746479
mean loss:  5.643981348723173
epoch:  4
train accuracy mean:  0.12277415442088295
validation accuracy:  0.10234741784037558
mean loss:  5.4218880993624525
epoch:  5
train accuracy mean:  0.1268339611235119
validation accuracy:  0.1123943661971831
mean loss:  5.221812829375267
epoch:  6
train accuracy mean:  0.13099355546254962
validation accuracy:  0.12065727699530517
mean loss:  5.040732581013192
epoch:  7
train accuracy mean:  0.1360991947

## Load Model

In [151]:
load_state = torch.load('embeddings_best_model')
model.load_state_dict(load_state['model'])
model.eval()
print('accuracy in val: ', eval_model(val_loader, model, use_gpu))

accuracy in val:  0.14178403755868543


In [152]:
NGramModel = NGramNeuralModel(ngram_builder, model)

## Generate Sequence

In [22]:
seq = NGramModel.generate_sequence('vete a la'.split(), use_gpu=use_gpu)
print_doc(seq)

['vete', 'a', 'la', 'verga', '🤔', '</s>']

In [23]:
seq = NGramModel.generate_sequence('chinga a tu '.split(), use_gpu=use_gpu)
print_doc(seq)

['chinga', 'a', 'tu', 'madre', 'mátenme', '</s>']

In [31]:
seq = NGramModel.generate_sequence('estoy a punto'.split(), use_gpu=use_gpu)
print_doc(seq)

estoy a punto vergazos este capítulo de como 💩 haz … <unk> a unos putos putos 


## Likelihood

In [34]:
NGramModel.estimate_prob('voy a estar en mi casa', use_gpu=use_gpu)

-12.998171

In [35]:
NGramModel.estimate_prob('voy a en estar mi casa', use_gpu=use_gpu)

-19.667858

In [36]:
NGramModel.estimate_prob('chinga a tu madre', use_gpu=use_gpu)

-0.023245202

In [41]:
NGramModel.estimate_prob('chinga a tu padre', use_gpu=use_gpu)

-9.934837

In [37]:
NGramModel.estimate_prob('a madre tu chinga', use_gpu=use_gpu)

-8.591314

## Permutations

In [42]:
test_structures('vas a chingar a tu madre'.split(), NGramModel)

a vas chingar a tu madre
likelihood:  -4.3084855

vas a chingar a tu madre
likelihood:  -5.482989

vas madre a chingar a tu
likelihood:  -7.2738085

chingar a tu madre vas a
likelihood:  -7.515975

vas a chingar tu madre a
likelihood:  -7.5483837

tu vas madre a chingar a
likelihood:  -7.9070125

tu madre vas a chingar a
likelihood:  -7.93376

a chingar tu madre vas a
likelihood:  -7.964194

a vas a chingar tu madre
likelihood:  -8.242043

a chingar vas a tu madre
likelihood:  -8.520243

a vas chingar tu madre a
likelihood:  -8.681716

a madre vas a chingar tu
likelihood:  -8.682941

madre a vas a chingar tu
likelihood:  -8.961501

vas chingar a tu madre a
likelihood:  -9.287172

madre tu vas a chingar a
likelihood:  -9.424377

a chingar vas tu madre a
likelihood:  -9.499789

a vas tu madre chingar a
likelihood:  -9.680814

vas tu madre a chingar a
likelihood:  -9.72591

a vas tu madre a chingar
likelihood:  -9.936181

madre tu chingar a vas a
likelihood:  -10.059231

chingar a vas a t

## Most Similar Words to Target Word

In [147]:
word = 'chinga'
indexes = most_similar_to(word, ngram_builder, model.embeddings.weight.detach().cpu().numpy(), 10)
print('Most Similar to: ', bold_string(word))
ngram_builder.inverse(indexes)

Most Similar to:  chinga 


['mentarte',
 'reputisima',
 'concha',
 'chingue',
 'put',
 'reputa',
 'putisima',
 'chingar',
 'chiga',
 'chingas']

In [52]:
word = 'amor'
indexes = most_similar_to(word, ngram_builder, model.embeddings.weight.detach().cpu().numpy(), 10)
print('Most Similar to: ', bold_string(word))
ngram_builder.inverse(indexes)

Most Similar to:  amor 


['llanto',
 'hombre',
 'pensamiento',
 'desprecio',
 'orgullo',
 'sufrimiento',
 'alma',
 'corazon',
 'corazón',
 'cariño']

In [53]:
word = 'verga'
indexes = most_similar_to(word, ngram_builder, model.embeddings.weight.detach().cpu().numpy(), 10)
print('Most Similar to: ', bold_string(word))
ngram_builder.inverse(indexes)

Most Similar to:  verga 


['berga',
 'caca',
 'fregada',
 'vergaaaaaa',
 'ñonga',
 'mierda',
 'chingada',
 'pija',
 'verg',
 'vrg']

## Cos Distance Among all Data

En esta sección obtendremos los 10 pares de tokens más similares entre sí entre todos los tokens luego de haber cargado los embeddings y haber realizado el entranamiento de la red.

In [45]:
def cos_distance(data):
    N = len(data)
    distances = np.zeros((N, N))
    magnitudes = np.linalg.norm(data, axis=1)
    
    for i in range(N):
        for j in range(i+1):
            distances[i, j] = np.dot(data[i], data[j])/(magnitudes[i] * magnitudes[j])
            if i != j:
                distances[j, i] = distances[i, j]
    
    return distances

In [46]:
def get_most_similar(dist_matrix, n):
    N = len(dist_matrix)
    
    # get indexes of elements to be compared. dist_matrix should be symmetric, so we dont need to consider each pair of distances twice
    indexes = [(i,j) for i in range(N) for j in range(i+1) if i!=j]

    # get x and y indexes
    x_indexes = tuple([ind[0] for ind in indexes])
    y_indexes = tuple([ind[1] for ind in indexes])
    
    # get values of matrix
    row_max = dist_matrix[x_indexes, y_indexes]
    
    # desc sort elements retrieved and get their positions
    max_elements = np.flip(np.argsort(row_max))[:n]
    
    # return indexes in positions retrieved in previous step
    return [indexes[max_index] for max_index in max_elements]

## Get Most Similar among all words

En los siguientes pares podemos ver cuales son los más parecidos de todas las palabras del vocabulario. Podríamos pensar que de hecho estamos obteniendo a la palabra consigo misma como par, pero de hecho en la implementación nos encargamos de evitar que eso sea posible, y si nos fijamos cuidadosamente las palabras a pesar de ser muy parecidas difieren en la cantidad de caracteres que tienen. Podemos ver entonces de esta forma y con las palabras más parecidas a una palabra en específico en la sección anterior que a pesar del entrenamiento, la red aún conserva la similitud a través de los embeddings en varias palabras. Pero, incluso con esto no obtenemos un mejor valor para la perplejidad ni la precisión en el conjunto de datos de validación.

In [47]:
dist_matrix = cos_distance(model.embeddings.weight.detach().cpu().numpy())

In [48]:
similar = get_most_similar(dist_matrix, 10)
similar = [list(pair) for pair in similar]
ngram_builder.inverse(similar)

[['goooooool', 'gooooool'],
 ['jajajajajajajajajaja', 'jajajajajajajajaja'],
 ['goooool', 'gooooool'],
 ['jajajajajajajajaja', 'jajajajajajajaja'],
 ['goooool', 'gooool'],
 ['goooooool', 'goooool'],
 ['jajajajajajajaja', 'jajajajajajaja'],
 ['jajajajajajajajajajaja', 'jajajajajajajajajaja'],
 ['jajajajajajaja', 'jajajajajaja'],
 ['yaaaaaa', 'yaaaaa']]

## Perplexity

In [153]:
NGramModel.perplexity(val_documents, use_gpu=use_gpu)

266.03431212853246

# Conclusión

A través de este ejercicio podemos resaltar varias cosas interesantes. Primero podemos comparar como con ambos modelos uno termina con un modelo de embeddings en donde las palabras más cercanas no parecen tener relación entre sí, y el otro (el modelo con embeddings pre-entrenados) definitivamente agrupa palabras similares entre sí al comparar a través de la distancia coseno. Esto es de esperar pues el modelo pre-entrenado ya tenía palabras que guardaban una similitud entre sí cercanas inicialmente. 

Sin embargo, esto no parece ayudar al modelo, pues el modelo sin embeddings pre-entrenados obtiene una mejor precisión en el conjunto de validación durante el entrenamiento, pero como dato curioso, la perplejidad del modelo es mayor con un valor de 353 en comparación con 266 para el modelo con embeddings pre-entrenados. Esto puede parecer contra intuitivo, pero una posible explicación podría ser el modelo con embeddings preentrenado en efecto asigna una mayor probabilidad de manera significativa al dataset de validación, aunque en cuanto a la cantidad el modelo sin embeddings preentrenados logra predecir de manera correcta más tokens, aunque la frontera de decisión sea mucho más parecida a otros tokens. La intuición de que este modelo pretende agrupar términos similares a través de sus embeddings y generalizar de esta manera palabras que tiene una relación semántica, podría tener lugar debido a que puede que la red al tener estos embeddings preentrenados asigne una mayor probabilidad a tokens con una semántica similar en función de los patrones que observo durante entrenamiento. Es decir, tal vez durante entrenamiento observó la secuencuia "veré a tu padre" y a través de los embeddings predice que la frase en validación "veré a tu madre" tiener mayor probabilidad que otras opciones y asigna una mayor probabilidad de manera significativa al token "madre" en comparación con el modelo sin embeddings preentrenados. 

Sin embargo, vale la pena mencionar que este resultado no se obtuvo de manera consistente todas las veces que se entrenó y experimentó con el modelo, había escenarios en los cuales la perplejidad del modelo sin embeddings pre-entrenados obtenía un menor valor que el de embeddings preentrenados. Esto podría sugerir la necesidad de muchos más datos para que las redes obtengan un comportamiento estable. 

Otro dato que también vale la pena mencionar, es que de manera consistente el modelo con embeddings preentrenados siempre requirió de menos iteraciones para terminar de ser entrenado, lo cual sugiere que los embeddings preentrenados si ayudan al modelo a converger más rápido a una solución probablemente local.

# Ejercicio 3

In [17]:
class BengioModel(nn.Module):
    def __init__(self, N, voc_size, d_model, hidden_size=128, emb_mat=None, dropout=0.1):
        
        super(BengioModel, self).__init__()
        # parameters
        self.N           = N
        self.d_model     = d_model
        self.voc_size    = voc_size
        self.hidden_size = hidden_size
        
        # Matriz entrenable de embeddings, tamaño vocab_size x Ngram.d_model
        self.embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(emb_mat), freeze=False)
        
        # fully connected layers
        self.fc1 = nn.Linear(d_model * (N-1), hidden_size)
        self.fc2 = nn.Linear(hidden_size, voc_size, bias=False)
        # direct connection to output
        self.W = nn.Linear(d_model * (N-1), voc_size, bias=False)
        
        # dropout
        self.drop = nn.Dropout(dropout)
        
    
    def forward(self, input_seq):
        # Calcula el embedding para cada palabra.
        x = self.embeddings(input_seq)
        x = x.view(-1, (self.N-1) * self.d_model)
        
        # direct connextion to output
        direct_link = self.W(x)
        # rest of bengio model
        x = self.fc1(x)
        x = self.drop(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x + direct_link

## Hyperparameters

In [25]:
ngram_builder = NGramBuilder(embeddings=embeddings)
ngram_docs, ngram_labels = ngram_builder.fit(documents, N=4)

In [26]:
# model hyperparameters
voc_size = ngram_builder.voc_size
N = ngram_builder.N
d_model = ngram_builder.d_model

# optimizer hyperparameters
lr = 2.3e-1 
epochs = 100
patience = epochs//5

# scheduler hyperparameters
lr_patience = 10
lr_factor = 0.5

# gpu available?
use_gpu = torch.cuda.is_available()

# build model and move to gpu if possible
model = BengioModel(N=N, voc_size=voc_size, d_model=d_model, hidden_size=200, emb_mat=ngram_builder.emb_matrix)
if use_gpu:
    model = model.cuda()
    
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer,
                'min',
                patience = lr_patience,
                verbose=True,
                factor = lr_factor
            )

criterion = nn.CrossEntropyLoss()

## Train Embeddings Model

In [27]:
best_metric = 0
last_metric = 0
val_metrics = []
counter = 0

for epoch in range(epochs):
    print('epoch: ', 1 + epoch)
    epoch_metrics = []
    epoch_losses = []
    model.train()
    for inputs, targets in train_loader:
        if use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()

        # feed model and get loss
        output = model(inputs)
        loss = criterion(output, targets)
        epoch_losses.append(loss.item())

        # metric with train dataset
        preds = get_preds(output)
        epoch_metrics.append(accuracy(preds, targets.cpu().numpy()))

        # step to optimize 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # close for each step

    # get metric for training set
    model.eval()
    train_acc = np.mean(epoch_metrics)
    val_acc = eval_model(val_loader, model, use_gpu)
    val_metrics.append(val_acc)

    # print metrics
    print('train accuracy mean: ', train_acc)
    print('validation accuracy: ', val_acc)
    print('mean loss: ', np.mean(epoch_losses))

    # store model if necessary
    state = {
                'epoch' : epoch + 1,
                'optimizer': optimizer.state_dict(),
                'model': model.state_dict(),
                'scheduler': scheduler.state_dict(),
                'best_metric': best_metric
            }
    checkpoint(state, 'dirlink_best_model', val_acc, best_metric)

    # patience and last_metric and best_metric update
    last_metric = val_acc
    counter = counter + 1 if last_metric <= best_metric else 0
    best_metric = val_acc if val_acc > best_metric else best_metric

    # check if patience run out
    if counter >= patience:
        break
# close for each epoch

epoch:  1
train accuracy mean:  0.07259211464533731
validation accuracy:  0.05314553990610329
mean loss:  9.010061198845506
Storing best model to dirlink_best_model. Current acc: 0.05314553990610329, last best metric: 0
epoch:  2
train accuracy mean:  0.08799331907242064
validation accuracy:  0.0787793427230047
mean loss:  7.38438277070721
Storing best model to dirlink_best_model. Current acc: 0.0787793427230047, last best metric: 0.05314553990610329
epoch:  3
train accuracy mean:  0.10860431005084326
validation accuracy:  0.08488262910798122
mean loss:  6.57156110368669
Storing best model to dirlink_best_model. Current acc: 0.08488262910798122, last best metric: 0.0787793427230047
epoch:  4
train accuracy mean:  0.13793073381696427
validation accuracy:  0.09539906103286384
mean loss:  6.046455083725353
Storing best model to dirlink_best_model. Current acc: 0.09539906103286384, last best metric: 0.08488262910798122
epoch:  5
train accuracy mean:  0.16436961340525794
validation accuracy

## Load Model

In [28]:
load_state = torch.load('dirlink_best_model')
model.load_state_dict(load_state['model'])
model.eval()
eval_model(val_loader, model, use_gpu)

0.11643192488262911

In [29]:
NGramModel = NGramNeuralModel(ngram_builder, model)

## Perplexity

In [30]:
NGramModel.perplexity(val_documents, use_gpu=use_gpu)

/home/juan/Documents/CIMAT/nlp/tareas/tarea5/NGrams.py:228: RuntimeWarning: divide by zero encountered in log
  log_perp = np.sum(-np.log(cond_probs))     # log(1/cond_probs) = log(1) - log(cond_probs) = -log(cond_probs)


inf

Podemos ver que el modelo devuelve una perplejidad de infinito, lo cual significa que alguno de los tokens como objetivo tiene una probabilidad de 0. Esto no es de extrañar ya que en otras experimentaciones se obtenía valores muy altos de perplejidad para este modelo, lo que parece indicar que había tokens objetivo que tenían probabilidades sumamente bajas. Entonces, de todos los tokens que no tenían una probabilidad de 0 se calculo el que tenía la menor probabilidad, obtuvimos un token que tenía una probabilidad de 4e-45. Entonces, sumamos un valor sumamente pequeño de 1e-60 a las probabilidades que tenían un valor de 0 y calculamos nuevamente las perplejidades en el siguiente bloque, y obtuvimos el valor que se muestra abajo.

In [33]:
ngrams, targets = ngram_builder.transform(val_documents)
ngrams = torch.tensor(ngrams)
if use_gpu:
    ngrams = ngrams.cuda()
logits = model(ngrams)
probs = get_probs(logits)

# get cond probs and perplexity
num_target = [i for i in range(len(targets))]
cond_probs = probs[num_target, targets]
np.min(cond_probs[cond_probs>0])
 

4e-45

In [34]:
mod_probs = cond_probs + (cond_probs==0) * (1e-60)
log_perp = np.sum(-np.log(mod_probs))     # log(1/cond_probs) = log(1) - log(cond_probs) = -log(cond_probs)
perp = np.exp(1/len(targets) * log_perp)
perp

33692.316505321425

In [94]:
print_doc(NGramModel.generate_sequence(use_gpu=use_gpu))

<s> <s> <s> mira ud tiene que entender que me <unk> <unk> <unk> <unk> <unk> en tu <unk> por <unk> que estaba todo 


In [95]:
print_doc(NGramModel.generate_sequence(use_gpu=use_gpu))

<s> <s> <s> 😄 por qué <unk> me <unk> fotos jajajaja 


In [168]:
print_doc(NGramModel.generate_sequence(use_gpu=use_gpu))

<s> <s> <s> no <unk> a mi escuela le chingas a tu madre <unk> a mi que yo <unk> de <unk> y un letrero aqui está valiendo madre por gastar dinero <unk> las <unk> y <unk> ahi valen verga como <unk> <unk> <unk> 


## Conclusión

Podemos ver que inicialmente la pérdida es mayor que para los otros dos modelos, esto probablemente se debe a que hay una mayor cantidad de parámetros que deben ser ajustados. Para este modelo se debe propagar el error obtenido en la salida a esta nueva matriz de parámetros, y podemos ver que el modelo requiere en general de más epocas para entrenarse. 

El modelo de hecho no logra obtener un mejor valor para la perplejidad que los dos modelos anteriores, y es de hecho una perplejidad mucho mayor que los otros modelos, incluso cuando no se obtenía valores de infinito, la perplejidad era del orden de 10e+4. A través de la generación de secuencias podemos ver que el modelo genera muchas veces el token UNK. Si bien la capa extra para conectar directamente los embeddings a la salida tal vez podría ser de utilidad, en este escenario y considerando que hay pocos datos relativamente para realizar el entrenamiento, esto nos lleva a pensar que no se puede ver el beneficio de estas conexiones directas, o bien podría ser que de manera general no ayudan ni aportan a la red información adicional para poder predecir el siguiente token.  